# Python for Data Science Project Session 4: Economics and Finance

This dataset contains the hourly and daily count of rental bikes between years 2011 and 2012 in the Capital bike-share system with the corresponding weather and seasonal information. More information about the dataset you can find [here](https://archive-beta.ics.uci.edu/ml/datasets/bike+sharing+dataset). This notebook will cover tasks such as data transformations, pivot tables and simple regression.

## Analysing the dataset

First, let's import Pandas and NumPy.

In [2]:
import pandas as pd
import numpy as np

Now, we need to upload the data (use `pandas.csv_read()`,dataset name is `day.csv`, and save if as `df`).

In [3]:
df=pd.read_csv('day.csv')

Display the dataframe and use `.describe()` to check if your dataset has any missing values.

In [4]:
df.describe()
# no missing values

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000,731.000000
mean,366.000000,2.496580,0.500684,6.519836,0.028728,2.997264,0.683995,1.395349,0.495385,0.474354,0.627894,0.190486,848.176471,3656.172367,4504.348837
std,211.165812,1.110807,0.500342,3.451913,0.167155,2.004787,0.465233,0.544894,0.183051,0.162961,0.142429,0.077498,686.622488,1560.256377,1937.211452
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.059130,0.079070,0.000000,0.022392,2.000000,20.000000,22.000000
25%,183.500000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,0.337083,0.337842,0.520000,0.134950,315.500000,2497.000000,3152.000000
50%,366.000000,3.000000,1.000000,7.000000,0.000000,3.000000,1.000000,1.000000,0.498333,0.486733,0.626667,0.180975,713.000000,3662.000000,4548.000000
75%,548.500000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,0.655417,0.608602,0.730209,0.233214,1096.000000,4776.500000,5956.000000
max,731.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,0.861667,0.840896,0.972500,0.507463,3410.000000,6946.000000,8714.000000


We can see that our dataset has no missing values. Now, let's drop columns that we won't use (`casual`, `registered`).

In [5]:
df.drop(['casual','registered'],axis=1)

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,727,2012-12-27,1,1,12,0,4,1,2,0.254167,0.226642,0.652917,0.350133,2114
727,728,2012-12-28,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,3095
728,729,2012-12-29,1,1,12,0,6,0,2,0.253333,0.242400,0.752917,0.124383,1341
729,730,2012-12-30,1,1,12,0,0,0,1,0.255833,0.231700,0.483333,0.350754,1796


As we can see our weekdays are displayed as numbers. We want to make it more intuitive, so that we could see the name of the day corresponding to the number. To do it, create the dataframe that contains the number (`no`; 0, 1, 2, ...) and the corresponding day (`day`; "Mon", "Tue", "Wed", ...). Call it `weekdays`.

In [6]:
weekdays = pd.DataFrame(data={'no': [0,1,2,3,4,5,6], 'day': ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']})


The last piece of data that we will be using is the data about shifts. `shift.csv` contains the date and the name of the employee that was on a shift that day (let's say they work at the helpdesk). We need to upload it (call the dataframe `shift`, first column `date` and the second one `employee`) and display it.

In [7]:
shift = pd.read_csv("shift.csv",header=None,names=['date','employee'])


In [8]:
shift

,date,employee
0,01/01/2011,Kate
1,02/01/2011,John
2,03/01/2011,Harry
3,04/01/2011,Harry
4,05/01/2011,John
...,...,...
726,27/12/2012,John
727,28/12/2012,Kate
728,29/12/2012,Kate
729,30/12/2012,Kate


We have all the data that we need!

We would like to combine `weekdays` with `df` on the number of the day. As we can see, in `df` the number of the day is called `weekday`, and in `weekdays` it is called `no`. Therefore, we need to change the name of one of the columns. Let's rename the `weekdays` dataframe column name from `no` to `weekday` (use `.rename()`).

In [9]:
weekdays=weekdays.rename(columns={'no':'weekday'})

Now we can merge them on `weekday` (use `.merge()`). Name the new dataframe `merged`.

In [10]:
merged=pd.merge(df,weekdays,how='inner',on='weekday')

Let's check if we merged the data cocrrectly.

In [11]:
merged

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,Sun
1,8,2011-01-08,1,0,1,0,6,0,2,0.165000,0.162254,0.535833,0.266804,68,891,959,Sun
2,15,2011-01-15,1,0,1,0,6,0,2,0.233333,0.248112,0.498750,0.157963,222,1026,1248,Sun
3,22,2011-01-22,1,0,1,0,6,0,1,0.059130,0.079070,0.400000,0.171970,93,888,981,Sun
4,29,2011-01-29,1,0,1,0,6,0,1,0.196522,0.212126,0.651739,0.145365,123,975,1098,Sun
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,700,2012-11-30,4,1,11,0,5,1,1,0.298333,0.323867,0.649583,0.058471,362,5306,5668,Sat
727,707,2012-12-07,4,1,12,0,5,1,2,0.320833,0.321958,0.764167,0.130600,349,4659,5008,Sat
728,714,2012-12-14,4,1,12,0,5,1,1,0.281667,0.294192,0.642917,0.131229,429,5182,5611,Sat
729,721,2012-12-21,1,1,12,0,5,1,2,0.326667,0.301767,0.556667,0.374383,221,3402,3623,Sat


We would like to do the same with our `merged` dataframe and `shift` dataframe. As in the pervious example, we need to rename some columns. Rename `dteday` to `date` and display the new `date` column.

In [12]:
merged=merged.rename(columns={'dteday':'date'})

In [13]:
merged['date']

0      2011-01-01
1      2011-01-08
2      2011-01-15
3      2011-01-22
4      2011-01-29
          ...    
726    2012-11-30
727    2012-12-07
728    2012-12-14
729    2012-12-21
730    2012-12-28
Name: date, Length: 731, dtype: object

As we can see, we have a different date formats. To fix it we going to use `datetime` library, `.strptime()` and `.strftime()`. You can find an example of how to do it [here](https://stackoverflow.com/questions/14524322/how-to-convert-a-date-string-to-different-format).

In [14]:
import datetime

In [15]:
for i in range(0,merged.shape[0]):
    merged['date'].iloc[i] =datetime.datetime.strptime(merged['date'].loc[i], '%Y-%m-%d').strftime('%d/%m/20%y')

/var/folders/g1/7qhqkzx5133glbzkt6wybybh0000gn/T/ipykernel_60696/2896019112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged['date'].iloc[i] =datetime.datetime.strptime(merged['date'].loc[i], '%Y-%m-%d').strftime('%d/%m/20%y')


After changing the date format, you can merge the two dataframes together. Name the final dataframe `final_df`.

In [16]:
final_df = pd.merge(merged,shift,how='inner',on='date')

To check if you have correctly merged the dataframe, display the sample of 10 rows from the `final_df`.

In [17]:
final_df.tail(10)

,instant,date,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,day,employee
721,665,26/10/2012,4,1,10,0,5,1,2,0.545833,0.522750,0.807083,0.132467,1182,6262,7444,Sat,Kate
722,672,02/11/2012,4,1,11,0,5,1,1,0.355000,0.356042,0.522083,0.266175,618,5229,5847,Sat,Kate
723,679,09/11/2012,4,1,11,0,5,1,1,0.361667,0.355413,0.540833,0.214558,709,5283,5992,Sat,Kate
724,686,16/11/2012,4,1,11,0,5,1,1,0.345000,0.347204,0.524583,0.171025,484,5214,5698,Sat,Kate
725,693,23/11/2012,4,1,11,0,5,1,1,0.368333,0.378779,0.568750,0.148021,1603,2307,3910,Sat,Kate
726,700,30/11/2012,4,1,11,0,5,1,1,0.298333,0.323867,0.649583,0.058471,362,5306,5668,Sat,Kate
727,707,07/12/2012,4,1,12,0,5,1,2,0.320833,0.321958,0.764167,0.130600,349,4659,5008,Sat,Kate
728,714,14/12/2012,4,1,12,0,5,1,1,0.281667,0.294192,0.642917,0.131229,429,5182,5611,Sat,Kate
729,721,21/12/2012,1,1,12,0,5,1,2,0.326667,0.301767,0.556667,0.374383,221,3402,3623,Sat,Kate
730,728,28/12/2012,1,1,12,0,5,1,2,0.253333,0.255046,0.590000,0.155471,644,2451,3095,Sat,Kate


Let's say that we want to inspect the employees performance. Display the mean `cnt` for each employee using `.groupby()`.

In [18]:
final_df['cnt'].groupby(final_df['employee']).mean()

employee
Harry    4374.960396
John     4520.156489
Kate     4586.726592
Name: cnt, dtype: float64

Harry has lower `cnt` compared to the others. It might be because they work on different days of the week. To check it, first let's check if the `cnt` differ across different days of the week. Display the mean `cnt` for each day of the week.

In [19]:
final_df['cnt'].groupby(final_df['day']).mean()

day
Fri    4667.259615
Mon    4228.828571
Sat    4690.288462
Sun    4550.542857
Thu    4548.538462
Tue    4338.123810
Wed    4510.663462
Name: cnt, dtype: float64

The differences in mean `cnt` across different days of the week do exist! To check if it causes Harry to has lower `cnt`, we can use `.pivot_table()`.

In [21]:
pivot = pd.pivot_table(final_df, values='cnt', index=['employee'],
                    columns=['day'], fill_value=0, aggfunc=np.mean)

In [22]:
pivot

day,Fri,Mon,Sat,Sun,Thu,Tue,Wed
employee,,,,,,,
Harry,0.000000,4353.680556,0.000000,0.000000,0.000000,4130.560606,4650.937500
John,4667.259615,3899.333333,0.000000,0.000000,4548.538462,4808.800000,3745.894737
Kate,0.000000,4004.000000,4690.288462,4550.542857,0.000000,4563.684211,4775.095238


As we can see, Harry works only on Monday, Tuesday and Wednesday, which might be the cause of his lower `cnt`.

# OLS model

Now we will create a simple predictive model, which will forecast the `cnt` for a given day. To do it, we need to import `statsmodels.api`.

In [33]:
import statsmodels.api as st

We can drop all of the unnecessary data, so that only `mnth`, `holiday`, `workingday`, `temp`, `atemp`, `hum`, `windspeed`, `day` and `cnt` are left.

In [42]:
final_df = final_df.loc[:,["mnth","holiday","workingday","temp","atemp","hum","windspeed","day","cnt"]]
final_df

,mnth,holiday,workingday,temp,atemp,hum,windspeed,day,cnt
0,1,0,0,0.344167,0.363625,0.805833,0.160446,Sun,985
1,1,0,0,0.165000,0.162254,0.535833,0.266804,Sun,959
2,1,0,0,0.233333,0.248112,0.498750,0.157963,Sun,1248
3,1,0,0,0.059130,0.079070,0.400000,0.171970,Sun,981
4,1,0,0,0.196522,0.212126,0.651739,0.145365,Sun,1098
...,...,...,...,...,...,...,...,...,...
726,11,0,1,0.298333,0.323867,0.649583,0.058471,Sat,5668
727,12,0,1,0.320833,0.321958,0.764167,0.130600,Sat,5008
728,12,0,1,0.281667,0.294192,0.642917,0.131229,Sat,5611
729,12,0,1,0.326667,0.301767,0.556667,0.374383,Sat,3623


The `day` is a categorical variable, so to run a regression we need to create dummy variables. To do it, use `.get_dummies()` command.

In [45]:
final_df=pd.get_dummies(final_df,columns=['day'])

Display the final_df to check if you have created the data correctly.

In [46]:
final_df

,mnth,holiday,workingday,temp,atemp,hum,windspeed,cnt,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed
0,1,0,0,0.344167,0.363625,0.805833,0.160446,985,0,0,0,1,0,0,0
1,1,0,0,0.165000,0.162254,0.535833,0.266804,959,0,0,0,1,0,0,0
2,1,0,0,0.233333,0.248112,0.498750,0.157963,1248,0,0,0,1,0,0,0
3,1,0,0,0.059130,0.079070,0.400000,0.171970,981,0,0,0,1,0,0,0
4,1,0,0,0.196522,0.212126,0.651739,0.145365,1098,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,11,0,1,0.298333,0.323867,0.649583,0.058471,5668,0,0,1,0,0,0,0
727,12,0,1,0.320833,0.321958,0.764167,0.130600,5008,0,0,1,0,0,0,0
728,12,0,1,0.281667,0.294192,0.642917,0.131229,5611,0,0,1,0,0,0,0
729,12,0,1,0.326667,0.301767,0.556667,0.374383,3623,0,0,1,0,0,0,0


Now it's time for the regression! Create two new dataframes `y` and `x`. `y` is the dataframe that contains the `cnt` column, and `x` contains all the other columns (of the dataframe with dummy variables).

In [84]:
y = pd.DataFrame(final_df.loc[:,'cnt'])
x = final_df.drop(['cnt'],axis=1)


,mnth,holiday,workingday,temp,atemp,hum,windspeed,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed
0,1,0,0,0.344167,0.363625,0.805833,0.160446,0,0,0,1,0,0,0
1,1,0,0,0.165000,0.162254,0.535833,0.266804,0,0,0,1,0,0,0
2,1,0,0,0.233333,0.248112,0.498750,0.157963,0,0,0,1,0,0,0
3,1,0,0,0.059130,0.079070,0.400000,0.171970,0,0,0,1,0,0,0
4,1,0,0,0.196522,0.212126,0.651739,0.145365,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,11,0,1,0.298333,0.323867,0.649583,0.058471,0,0,1,0,0,0,0
727,12,0,1,0.320833,0.321958,0.764167,0.130600,0,0,1,0,0,0,0
728,12,0,1,0.281667,0.294192,0.642917,0.131229,0,0,1,0,0,0,0
729,12,0,1,0.326667,0.301767,0.556667,0.374383,0,0,1,0,0,0,0


Now, we will run our model and display the model summary! (Just run the commands below).

In [69]:
model = st.OLS(y, x).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.487
Method:                 Least Squares   F-statistic:                     58.85
Date:                Thu, 09 Mar 2023   Prob (F-statistic):           2.57e-98
Time:                        18:50:13   Log-Likelihood:                -6319.3
No. Observations:                 731   AIC:                         1.266e+04
Df Residuals:                     718   BIC:                         1.272e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
mnth          95.5124     15.736      6.070      0.000      64.618     126.407
holiday     2023.4142    360.641      5.611      0.000    1315.378    2731.451
workingday  2648.9234    260.542     10.167      0.000    2137.409    3160.438
temp        1999.3621   2234.726      0.895      0.371   -2388.016    6386.740
atemp       4873.2270   2524.706      1.930      0.054     -83.462    9829.916
hum        -3513.7122    381.380     -9.213      0.000   -4262.465   -2764.960
windspeed  -3961.2880    710.284     -5.577      0.000   -5355.770   -2566.806
day_Fri      944.8725    162.756      5.805      0.000     625.338    1264.407
day_Mon     3328.1061    372.405      8.937      0.000    2596.973    4059.239
day_Sat     1033.4195    159.833      6.466      0.000     719.623    1347.216
day_Sun     3664.8581    372.634      9.835      0.000    2933.276    4396.440
day_Thu      939.2699    167.318      5.614      0.000     610.778    1267.761
day_Tue      854.5883    160.066      5.339      0.000     540.335    1168.842
day_Wed      900.1874    168.723      5.335      0.000     568.937    1231.438
==============================================================================
Omnibus:                        5.688   Durbin-Watson:                   0.682
Prob(Omnibus):                  0.058   Jarque-Bera (JB):                4.115
Skew:                           0.011   Prob(JB):                        0.128
Kurtosis:                       2.633   Cond. No.                     1.87e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 1.16e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

We can see all the important regression information which we can analyse!

To predict the value for the next day, we need to create a new dataframe that we will use as an input. Create a new datafeame `to_predict` with the same column names as `x` dataframe (you can use `.columns()`).

In [102]:
to_predict=pd.DataFrame(columns=x.columns)

Now lets append our dataframe with tomorrow's data which are as follows:

    Month: 1; Holiday: 0; Workingday: 1; Temp: 0.25; Atemp: 0.2; Hum: 0.5; Windspeed: 0.15; Day: Sat (you need to represent day as a set of dummy variables)

In [103]:
to_predict = to_predict.append({"mnth":1,"holiday":0,"workingday":1,"temp":0.25,"atemp":0.2,"hum":0.5,"windspeed":0.15,"day_Mon":0,"day_Tue":0,"day_Wed":0,"day_Thu":0,"day_Fri":0,"day_Sat":1,"day_Sun":0},ignore_index = True)

to_predict

/var/folders/g1/7qhqkzx5133glbzkt6wybybh0000gn/T/ipykernel_60696/382126271.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  to_predict = to_predict.append({"mnth":1,"holiday":0,"workingday":1,"temp":0.25,"atemp":0.2,"hum":0.5,"windspeed":0.15,"day_Mon":0,"day_Tue":0,"day_Wed":0,"day_Thu":0,"day_Fri":0,"day_Sat":1,"day_Sun":0},ignore_index = True)


,mnth,holiday,workingday,temp,atemp,hum,windspeed,day_Fri,day_Mon,day_Sat,day_Sun,day_Thu,day_Tue,day_Wed
0,1.0,0.0,1.0,0.25,0.2,0.5,0.15,0.0,0.0,1.0,0.0,0.0,0.0,0.0


To predict our dataframe, we just need to use `model.predict()` and as an argument plug in the dataframe with our values!

In [104]:
prediction =model.predict(to_predict)

In [105]:
prediction

0    2901.291922
dtype: float64